# GAN met Fashion MNIST

In deze notebook gaan we opnieuw beelden genereren op basis van de Fashin MNIST dataset, net zoals in de vorige notebook over variational autoencoders (VAE). 
Hierbij moeten we een generator en discriminator opstellen.

## Importeren van packages en dataset

Eerst importeren we alle benodigde Python-bibliotheken voor het bouwen, trainen en visualiseren van onze VAE.
We gebruiken Pytorch voor het bouwen van het neurale netwerk, matplotlib voor visualisaties en NumPy voor numerieke berekeningen.
Daarna laden we de Fashion MNIST dataset, normaliseren de pixelwaarden naar de range [-1,1] 
en splitsen de dataset in een trainings- en testset. We gebruiken DataLoader om mini-batches te maken voor training.

In [2]:
# Importeren van benodigde bibliotheken
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy as np

# Controleer of er een GPU beschikbaar is, zo niet gebruik de CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Data-transformatie: normaliseer de afbeeldingen zodat de pixelwaarden tussen -1 en 1 liggen
transform = transforms.Compose([
    transforms.ToTensor(),  # Converteert beeld naar tensor
    transforms.Normalize((0,), (1,))  # Normaliseert naar bereik [-1, 1]
])

# FashionMNIST dataset downloaden en laden
train_dataset = datasets.FashionMNIST(root='./data', train=True, transform=transform, download=True)

# DataLoader voor batches van de trainingsdata
train_loader = DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)

print("Data geladen en DataLoader klaar.")

Data geladen en DataLoader klaar.


## Generator

 Deze cel definieert de generator die een random vector van ruis (latent vector) gebruikt om een afbeelding van 28x28 pixels te genereren. Het netwerk bestaat uit vier volledig verbonden lagen met ReLU-activatie, gevolgd door een Tanh activatie om de output te normaliseren.

In [3]:
class Generator(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(Generator, self).__init__()

        self.model = nn.Sequential(
            nn.Linear(input_dim, 256),
            nn.ReLU(),
            nn.Linear(256, 512),
            nn.ReLU(),
            nn.Linear(512, 1024),
            nn.ReLU(),
            nn.Linear(1024, output_dim),
            nn.Tanh() # zodat de outputwaarden tussen -1 en 1 liggen
        )

    def forward(self, x):
        return self.model(x)

latent_dim = 100
generator = Generator(100, 28*28)
print(generator)

Generator(
  (model): Sequential(
    (0): Linear(in_features=100, out_features=256, bias=True)
    (1): ReLU()
    (2): Linear(in_features=256, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=1024, bias=True)
    (5): ReLU()
    (6): Linear(in_features=1024, out_features=784, bias=True)
    (7): Tanh()
  )
)


## Discriminator

Deze cel definieert de discriminator, die een afbeelding van 28x28 pixels als invoer ontvangt en een enkele waarde teruggeeft die aangeeft of de afbeelding echt is (uit de dataset) of vals (gegenereerd door de generator). Het netwerk bestaat uit vier volledig verbonden lagen met LeakyReLU-activatie en dropout om overfitting te voorkomen.

In [4]:
class Discriminator(nn.Module):
    def __init__(self, input_dim):
        super(Discriminator, self).__init__()

        self.model = nn.Sequential(
            nn.Linear(input_dim, 1024),
            nn.LeakyReLU(),
            nn.Dropout(0.3),
            nn.Linear(1024,512),
            nn.LeakyReLU(),
            nn.Dropout(0.3),
            nn.Linear(512,256),
            nn.LeakyReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.model(x)

discriminator = Discriminator(28*28)
print(discriminator)

Discriminator(
  (model): Sequential(
    (0): Linear(in_features=784, out_features=1024, bias=True)
    (1): LeakyReLU(negative_slope=0.01)
    (2): Dropout(p=0.3, inplace=False)
    (3): Linear(in_features=1024, out_features=512, bias=True)
    (4): LeakyReLU(negative_slope=0.01)
    (5): Dropout(p=0.3, inplace=False)
    (6): Linear(in_features=512, out_features=256, bias=True)
    (7): LeakyReLU(negative_slope=0.01)
    (8): Dropout(p=0.3, inplace=False)
    (9): Linear(in_features=256, out_features=1, bias=True)
    (10): Sigmoid()
  )
)


## Loss functions

In deze cel worden de verliesfunctie en de optimalizers voor de generator en discriminator gedefinieerd. We gebruiken binaire cross-entropy (BCELoss) als verliesfunctie en de Adam-optimizer voor zowel de generator als de discriminator.

In [9]:
criterion = nn.BCELoss() # loss function voor de descriminator

optimizer_D = optim.Adam(discriminator.parameters(), lr=0.002)
optimizer_G = optim.Adam(generator.parameters(), lr=0.002)

## Trainen van het GAN model

De discriminator wordt getraind met zowel echte als gegenereerde afbeeldingen om te leren onderscheid te maken tussen de twee, terwijl de generator wordt getraind om betere afbeeldingen te genereren die de discriminator niet kan onderscheiden van echte. Voor elke epoch printen we het verlies van zowel de generator als de discriminator.

In [12]:
num_epochs = 10
for epoch in range(num_epochs):
    for n, (real_images, _) in enumerate(train_loader):
        batch_size = real_images.size(0)
        # Train Discriminator - real images
        real_images = real_images.view(batch_size, -1) # flatten de images
        real_labels = torch.ones(batch_size, 1) # elke figuur heeft klasse 1 = echt
        outputs = discriminator(real_images)
        loss_real = criterion(outputs, real_labels)
        optimizer_D.zero_grad()
        loss_real.backward()

        # Train Discriminator - fake images
        noise = torch.randn(batch_size, latent_dim) # ruis met dimensie (batchsize, latent_dim)
        fake_images = generator(noise)
        fake_labels = torch.zeros(batch_size, 1) # elke figuur heeft klasse 0 = fake
        outputs = discriminator(fake_images)
        loss_fake = criterion(outputs, fake_labels)
        loss_fake.backward()
        
        optimizer_D.step() # na deze stap heeft de discriminator een optimalisatie uitgevoerd

        loss_D = loss_real + loss_fake

        # Train generator
        outputs = discriminator(fake_images)
        loss = criterion(outputs, real_labels) # hij had eigenlijk moeten voorspellen dat het echte waren
        optimizer_G.zero_grad()
        loss.backward()
        optimizer_G.step()

    print(f"Epoch {epoch}/{num_epochs}")

        
    

RuntimeError: Trying to backward through the graph a second time (or directly access saved tensors after they have already been freed). Saved intermediate values of the graph are freed when you call .backward() or autograd.grad(). Specify retain_graph=True if you need to backward through the graph a second time or if you need to access saved tensors after calling backward.

## Visualiseren van de resultaten

Deze cel definieert een functie om enkele gegenereerde afbeeldingen van de getrainde generator te visualiseren. Het genereert willekeurige ruis en laat de generator nieuwe afbeeldingen maken, die vervolgens worden weergegeven met behulp van Matplotlib.